In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.datasets import cifar100
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Activation,Input,UpSampling2D
from keras.models import Model,Sequential
from keras.layers import LeakyReLU,PReLU
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))
import tensorflow as tf
from keras.callbacks import EarlyStopping
# Any results you write to the current directory are saved as output.

In [ ]:
(xtrain,ytrain),(xtest,ytest)=cifar100.load_data()

In [ ]:
img_shape=xtrain.shape[1:]
N=xtrain.shape[0]


In [ ]:
plt.imshow(xtrain[0])

In [ ]:
#Data preprocessing
xtrain1=xtrain.astype('float32')/255
xtest1=xtest.astype('float32')/255

In [ ]:
inp=Input(shape=img_shape,dtype=tf.float32)
x=Conv2D(8,(3,3),padding='same')(inp)
x=Conv2D(8,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU(0.2)(x)
x=MaxPooling2D((2,2))(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU(0.2)(x)
encod=MaxPooling2D((2,2))(x)

x=Conv2D(8,(3,3),padding='same')(encod)
x=Conv2D(8,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU(0.2)(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU(0.2)(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(32,(3,3),padding='same')(x)
x=Conv2D(32,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU(0.2)(x)
x=Conv2D(3,(3,3),padding='same')(x)
x=Activation('sigmoid')(x)

In [ ]:
Encoder=Model(inp,encod)
Autoencoder=Model(inp,x)

In [ ]:
Autoencoder.summary()

In [ ]:
Encoder.summary()

In [ ]:
Autoencoder.compile(optimizer='Adam',loss='binary_crossentropy')


In [ ]:
earlystop=EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

In [ ]:
batchsize=64
epochs=30


In [ ]:
history=Autoencoder.fit(xtrain1,xtrain1,batch_size=batchsize,epochs=epochs,verbose=1,callbacks=[earlystop],validation_data=(xtest1,xtest1),shuffle=True)

In [ ]:
#Testing the quality of images produced
def show_both_images(orig,dec,num=10):
    n=num
    plt.figure(figsize=(20,4))
    
    for i in range(n):
        ax=plt.subplot(2,n,i+1)
        plt.imshow(orig[i].reshape(32,32,3))
        
        ax=plt.subplot(2,n,i+1+n)
        plt.imshow(dec[i].reshape(32,32,3))
    plt.show()


In [ ]:
cifar_test=Autoencoder.predict(xtest1)

In [ ]:
show_both_images(xtest,cifar_test)

**# Trying New Idea
# **Using Parametric Activation function**
**

In [ ]:
inp=Input(shape=img_shape,dtype=tf.float32)
x=Conv2D(8,(3,3),padding='same')(inp)
x=Conv2D(8,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=PReLU()(x)
x=MaxPooling2D((2,2))(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=PReLU()(x)
encod=MaxPooling2D((2,2))(x)

x=Conv2D(8,(3,3),padding='same')(encod)
x=Conv2D(8,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=PReLU()(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=Conv2D(16,(3,3),padding='same')(x)
x=PReLU()(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(32,(3,3),padding='same')(x)
x=Conv2D(32,(3,3),padding='same')(x)
x=BatchNormalization()(x)
x=PReLU()(x)
x=Conv2D(3,(3,3),padding='same')(x)
x=Activation('sigmoid')(x)

In [ ]:
Encoder=Model(inp,encod)
Autoencoder=Model(inp,x)

In [ ]:
Encoder.summary() 
#alpha is initialized for each parameter

In [ ]:
Autoencoder.summary()

In [ ]:
earlystop=EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto')

In [ ]:
batchsize=64
epochs=30
Autoencoder.compile(optimizer='Adam',loss='binary_crossentropy')

In [ ]:
history=Autoencoder.fit(xtrain1,xtrain1,batch_size=batchsize,epochs=epochs,verbose=1,callbacks=[earlystop],validation_data=(xtest1,xtest1),shuffle=True)

In [ ]:
cifar_valid=Autoencoder.predict(xtest1)

In [ ]:
show_both_images(xtest,cifar_valid)

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
Autoencoder.layers[4].get_weights()
#list of Alpha values which model learnt

In [ ]:
print(Autoencoder.layers[4])